In [47]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "admin@123",
                               database = "datastorage")

cursor = conn.cursor()

In [8]:
df = pd.read_csv('product_cleaned.csv')
df.head()

,productkey,product,standard_cost,color,subcategory,category
0,210,"HL Road Frame - Black, 58",868.63,Black,Road Frames,Components
1,215,"Sport-100 Helmet, Black",12.03,Black,Helmets,Accessories
2,216,"Sport-100 Helmet, Black",13.88,Black,Helmets,Accessories
3,217,"Sport-100 Helmet, Black",13.09,Black,Helmets,Accessories
4,253,"LL Road Frame - Black, 58",176.20,Black,Road Frames,Components


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   productkey     397 non-null    int64  
 1   product        397 non-null    object 
 2   standard_cost  397 non-null    float64
 3   color          341 non-null    object 
 4   subcategory    397 non-null    object 
 5   category       397 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 18.7+ KB


In [18]:
df['color'] = df['color'].fillna("unknown")

In [20]:
df[df['productkey'] == 210]

,productkey,product,standard_cost,color,subcategory,category
0,210,"HL Road Frame - Black, 58",868.63,Black,Road Frames,Components


In [ ]:
df['productkey'].nunique()

397

In [23]:
df.shape

(397, 6)

In [9]:
df.columns

Index(['productkey', 'product', 'standard_cost', 'color', 'subcategory',
       'category'],
      dtype='object')

In [13]:
create_product_table = """CREATE TABLE products (
    productkey INT PRIMARY KEY,
    product VARCHAR(255),
    standard_cost DECIMAL(10,2),
    color VARCHAR(50),
    subcategory VARCHAR(100),
    category VARCHAR(100)
);"""

In [14]:
cursor.execute(create_product_table)
conn.commit()

In [ ]:
# Insert data row by row
for _, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO products (productkey, product, standard_cost, color, subcategory, category)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (row["productkey"], 
         row["product"], 
         row["standard_cost"], 
         row["color"], 
         row["subcategory"], 
         row["category"])
    )

# Commit and close connection
conn.commit()
print("CSV data imported successfully!")

CSV data imported successfully!


In [51]:
df_region = pd.read_csv('region_cleaned.csv')

In [52]:
df_region.head()

,salesterritorykey,region,country,country_group
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America


In [54]:
df_region['country_group'].value_counts()

country_group
North America    6
Europe           3
Pacific          1
Name: count, dtype: int64

In [44]:
df_region.rename(columns={"group": "country_group"}, inplace=True)

In [45]:
df_region.to_csv('region_cleaned.csv', index=False)

In [ ]:
Region_table_query = """CREATE TABLE region (
    salesterritorykey INT PRIMARY KEY,
    region VARCHAR(50) NOT NULL,
    country VARCHAR(50) NOT NULL,
    group VARCHAR(50) NOT NULL
);
"""
cursor.execute(Region_table_query)
conn.commit()

In [43]:
conn.close()

In [48]:
for _, row in df_region.iterrows():
    cursor.execute(
        """
        INSERT INTO region (salesterritorykey, region, country, country_group)
        VALUES (%s, %s, %s, %s)
        """,
        (row["salesterritorykey"], 
         row["region"], 
         row["country"], 
         row["country_group"])
    )

# Commit and close connection
conn.commit()
print("CSV data imported successfully!")

CSV data imported successfully!


In [49]:
df_reseller = pd.read_csv('reseller_cleaned.csv')
df_reseller.head()

,resellerkey,business_type,reseller,city,state_province,country_region
0,1,Value Added Reseller,A Bike Store,Seattle,Washington,United States
1,2,Specialty Bike Shop,Progressive Sports,Renton,Washington,United States
2,3,Warehouse,Advanced Bike Components,Irving,Texas,United States
3,4,Value Added Reseller,Modular Cycle Systems,Austin,Texas,United States
4,5,Specialty Bike Shop,Metropolitan Sports Supply,Fremont,California,United States


In [55]:
reseller_table_query = """CREATE TABLE reseller (
    resellerkey INT PRIMARY KEY,         -- Unique key for each reseller
    business_type VARCHAR(255) NOT NULL, -- Type of business
    reseller VARCHAR(255) NOT NULL,      -- Reseller name
    city VARCHAR(100) NOT NULL,          -- City where reseller is located
    state_province VARCHAR(100) NOT NULL, -- State or province
    country_region VARCHAR(100) NOT NULL -- Country or region
);
"""
cursor.execute(reseller_table_query)
conn.commit()

In [56]:
for _, row in df_reseller.iterrows():
    cursor.execute(
        """
        INSERT INTO reseller (resellerkey, business_type, reseller, city, state_province, country_region)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (row["resellerkey"], 
         row["business_type"], 
         row["reseller"], 
         row["city"], 
         row["state_province"], 
         row["country_region"])
    )

# Commit and close connection
conn.commit()
print("CSV data imported successfully!")


CSV data imported successfully!


In [58]:
df_sales = pd.read_csv('sales_cleaned.csv')
df_sales.head()

,salesordernumber,orderdate,productkey,resellerkey,employeekey,salesterritorykey,quantity,unit_price,sales,cost
0,SO43897,2017-08-25,235,312,282,4,2,28.84,57.68,63.45
1,SO43897,2017-08-25,351,312,282,4,2,2024.99,4049.98,3796.19
2,SO43897,2017-08-25,348,312,282,4,2,2024.99,4049.98,3796.19
3,SO43897,2017-08-25,232,312,282,4,2,28.84,57.68,63.45
4,SO44544,2017-11-18,292,312,282,4,2,818.70,1637.40,1413.62


In [59]:
df_salesperson = pd.read_csv('salesperson_cleaned.csv') 
df_salesperson.head()

,employeekey,employeeid,salesperson,title,upn
0,272,502097814,Stephen Jiang,North American Sales Manager,stephen-jiang@adventureworks.com
1,277,112432117,Brian Welcker,Director of Sales,brian-welcker@adventureworks.com
2,281,841560125,Michael Blythe,Sales Representative,michael-blythe@adventureworks.com
3,282,191644724,Linda Mitchell,Sales Representative,linda-mitchell@adventureworks.com
4,283,615389812,Jillian Carson,Sales Representative,jillian-carson@adventureworks.com


In [61]:
create_employee_table = """CREATE TABLE employee (
    employeekey INT PRIMARY KEY,          -- Unique identifier for employees
    employeeid BIGINT UNIQUE NOT NULL,    -- Employee ID (large integer)
    salesperson VARCHAR(100) NOT NULL,    -- Name of the salesperson
    title VARCHAR(100) NOT NULL,          -- Job title
    upn VARCHAR(255) UNIQUE NOT NULL      -- Email address (User Principal Name)
);
"""
cursor.execute(create_employee_table)
conn.commit()

In [62]:
for _, row in df_salesperson.iterrows():
    cursor.execute(
        """
        INSERT INTO employee (employeekey, employeeid, salesperson, title, upn)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (row["employeekey"], 
         row["employeeid"], 
         row["salesperson"], 
         row["title"], 
         row["upn"])
    )

# Commit and close connection
conn.commit()
print("CSV data imported successfully!")


CSV data imported successfully!
